In [75]:
import os, json
import pandas as pd

# this finds our json files
path_to_json = 'F:/kalu/BITS/DeepLearning/assignment1/CORD/train/json/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = pd.DataFrame(columns=['validline'])

f = open("F:/kalu/BITS/DeepLearning/assignment1/receipt_words.csv", "a")
print("filename",",","index",",","x1",",",'x2',",",'x3',",",'x4',",",'y1',",",'y2',",",'y3',",",'y4',",",'is_key',",",'row_id',",",'text',",",'category',",","group_id",file=f)


# we need both the json and an index number so use enumerate()
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)
        
       # print(json_text)

        # here you need to know the layout of your json and each json has to have
        # the same structure (obviously not the structure I have here)
        lines = json_text['valid_line']
        
        for x in lines:
            words = x['words']
            category = x['category']
            group_id = x['group_id']
            for y in words:
                print(js,index,y['quad']['x1'],y['quad']['x2'],y['quad']['x3'],y['quad']['x4'],y['quad']['y1'],y['quad']['y2'],y['quad']['y3'],y['quad']['y4'],y['is_key'],y['row_id'],"\""+y['text']+"\"",category,group_id,file=f,sep=",")

       # city = json_text['features'][0]['properties']['name']
        #lonlat = json_text['features'][0]['geometry']['coordinates']
        # here I push a list of data into a pandas DataFrame at row given by 'index'
        #f.close()
        #jsons_data.loc[index] = [words, city, lonlat]

# now that we have the pertinent json data in our DataFrame let's look at it
f.close()

In [1]:
import pandas as pd
receipts_df = pd.read_csv("F:/kalu/BITS/DeepLearning/assignment1/receipt_words.csv")
receipts_df.columns = receipts_df.columns.str.strip()
receipts_df['text'] = receipts_df['text'].str.strip()


In [2]:
receipts_df.dtypes

filename    object
index        int64
x1           int64
x2           int64
x3           int64
x4           int64
y1           int64
y2           int64
y3           int64
y4           int64
is_key       int64
row_id       int64
text        object
category    object
group_id     int64
dtype: object

In [3]:
char2idx = {"START":0,"END":1,"a":2,"b":3,"c":4,"d":5,"e":6,"f":7,"g":8,"h":9,"i":10,"j":11,"k":12,"l":13,"m":14,"n":15,"o":16,"p":17,"q":18,"r":19,"s":20,"t":21,"u":22,"v":23,"w":24,"x":25,"y":26,"z":27,"0":28,"1":29,"2":30,"3":31,"4":32,"5":33,"6":34,"7":35,"8":36,"9":37,".":38,"-":39,",":40,":":41," ":42}

In [4]:
#p_test.SentimentText=p_test.SentimentText.astype(str)
receipts_df.text = receipts_df.text.astype(str)
receipts_df.dtypes

filename    object
index        int64
x1           int64
x2           int64
x3           int64
x4           int64
y1           int64
y2           int64
y3           int64
y4           int64
is_key       int64
row_id       int64
text        object
category    object
group_id     int64
dtype: object

In [5]:
words = receipts_df['text']

In [6]:
indexed_words = []
j=43
for word in words:
    indexed_word = []
    #print(word)
    word = word.lower();
    for i in range(len(word)):
        token = word[i]
        if token not in char2idx:
            char2idx[token] = j
            j += 1
        indexed_word.append(char2idx[token])
    indexed_words.append(indexed_word)

receipts_df["indexed_words"] = indexed_words;

In [7]:
import numpy as np
def get_bigram_probs(words, V, start_idx, end_idx, smoothing=1):
  # structure of bigram probability matrix will be:
  # (last word, current word) --> probability
  # we will use add-1 smoothing
  # note: we'll always ignore this from the END token
  bigram_probs = np.ones((V, V)) * smoothing
  for word in words:
    for i in range(len(word)):
      
      if i == 0:
        # beginning word
        word[i]
        bigram_probs[start_idx, word[i]] += 1
      else:
        # middle word
        bigram_probs[word[i-1], word[i]] += 1

      # if we're at the final word
      # we update the bigram for last -> current
      # AND current -> END token
      if i == len(word) - 1:
        # final word
        bigram_probs[word[i], end_idx] += 1

  # normalize the counts along the rows to get probabilities
  bigram_probs /= bigram_probs.sum(axis=1, keepdims=True)
  return bigram_probs

In [8]:
start_idx = char2idx['START']
end_idx = char2idx['END']
V = len(char2idx)



In [9]:
bigram_probs = get_bigram_probs(indexed_words, V, start_idx, end_idx, smoothing=0.1)

In [10]:
bigram_probs

array([[5.27301010e-06, 5.27301010e-06, 1.57715732e-02, ...,
        5.27301010e-06, 1.10733212e-04, 1.10733212e-04],
       [1.53846154e-02, 1.53846154e-02, 1.53846154e-02, ...,
        1.53846154e-02, 1.53846154e-02, 1.53846154e-02],
       [1.57915515e-05, 7.97631267e-02, 8.05369128e-04, ...,
        1.57915515e-05, 1.57915515e-05, 1.57915515e-05],
       ...,
       [1.17647059e-02, 1.17647059e-02, 1.17647059e-02, ...,
        1.17647059e-02, 1.17647059e-02, 1.17647059e-02],
       [1.17647059e-02, 2.47058824e-01, 1.17647059e-02, ...,
        1.17647059e-02, 1.17647059e-02, 1.17647059e-02],
       [1.17647059e-02, 1.17647059e-02, 1.29411765e-01, ...,
        1.17647059e-02, 1.17647059e-02, 1.17647059e-02]])

In [11]:
 # a function to calculate normalized log prob score
  # for a sentence
def get_score(word):
    #print("word")
    #print(word)
    score = 0
    if len(word) > 0 :
        for i in range(len(word)):
          if i == 0:
            # beginning word
            score += np.log(bigram_probs[start_idx, word[i]])
          else:
            # middle word
            score += np.log(bigram_probs[word[i-1], word[i]])
        # final word
        score += np.log(bigram_probs[word[i-1], end_idx])

    # normalize the score
    # return score / (len(word) + 1)
    if score == 0:
        return score;
    else :
        return  ((-1)*(len(word) + 1)/score)*100

In [12]:
from future.utils import iteritems

idx2char = dict((v, k) for k, v in iteritems(char2idx))
def get_word(word):
    return ''.join(idx2char[i] for i in word)

In [13]:
receipts_df['text_score'] = receipts_df.apply(lambda row : get_score(row['indexed_words']), axis = 1)

In [14]:
receipts_df

,filename,index,x1,x2,x3,x4,y1,y2,y3,y4,is_key,row_id,text,category,group_id,indexed_words,text_score
0,receipt_00000.json,0,256,270,270,256,374,374,390,390,0,2179893,x,menu.cnt,3,[25],39.091172
1,receipt_00000.json,0,258,270,270,258,402,402,418,418,0,2179894,x,menu.cnt,4,[25],39.091172
2,receipt_00000.json,0,258,272,272,258,428,428,444,444,0,2179895,x,menu.cnt,5,[25],39.091172
3,receipt_00000.json,0,260,274,274,260,456,456,470,470,0,2179896,x,menu.cnt,6,[25],39.091172
4,receipt_00000.json,0,258,274,274,258,480,480,496,496,0,2179897,x,menu.cnt,7,[25],39.091172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18953,receipt_00799.json,786,1002,1179,1180,1002,1465,1465,1531,1528,0,678299,18.000,total.total_price,4,"[29, 36, 38, 28, 28, 28]",73.278820
18954,receipt_00799.json,786,274,415,414,274,1616,1618,1685,1683,1,678300,Tunai,total.cashprice,4,"[21, 22, 15, 2, 10]",35.914549
18955,receipt_00799.json,786,1002,1173,1176,1002,1621,1621,1685,1686,0,678300,19.000,total.cashprice,4,"[29, 37, 38, 28, 28, 28]",64.601556
18956,receipt_00799.json,786,268,474,475,268,1693,1695,1762,1761,1,678301,Kembali,total.changeprice,4,"[12, 6, 14, 3, 2, 13, 10]",49.430316


In [15]:
categories = receipts_df['category'] 
category2idx = {}
idx2category = {}
j=0;
for category in categories:
    if category not in category2idx:
        category2idx[category] = j
        idx2category[j] = category
        j+=1

category2idx

def get_index_for_category(category):
    return category2idx[category]


def get_category_for_index(index):
    return idx2category[index]
        

In [16]:
receipts_df['category_idx'] = receipts_df.apply(lambda row : get_index_for_category(row['category']), axis = 1)

In [18]:
data = receipts_df[['x1','x2','x3','x4','y1','y2','y3','y4','is_key','text_score','category_idx','index']]
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
x = data.values #returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = preprocessing.StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
print(x_scaled.shape)
X = pd.DataFrame(x_scaled,columns=['x1','x2','x3','x4','y1','y2','y3','y4','is_key','text_score','category_idx','index'])
X
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)

from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance

def getcosine(X,Y):
    return cosine_similarity(X,Y)[0,0];

def getneighourscore(index,X):
    return X['text_score'].tolist()[0];

X["n1_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,1]]]), axis = 1)
print("n1_cd done")
X["n2_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,2]]]), axis = 1)
print("n2_cd done")
X["n3_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,3]]]), axis = 1)
print("n3_cd done")
X["n4_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,4]]]), axis = 1)
print("n4_cd done")
X["n5_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,5]]]), axis = 1)
print("n5_cd done")

X["n1_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,1]]]), axis = 1)
print("n1_ed done")
X["n2_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,2]]]), axis = 1)
print("n2_ed done")
X["n3_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,3]]]), axis = 1)
print("n3_ed done")
X["n4_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,4]]]), axis = 1)
print("n3_ed done")
X["n5_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,5]]]), axis = 1)
print("n5_ed done")

X["n1_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,1]]]), axis = 1)
print("n1_score done")
X["n2_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,2]]]), axis = 1)
print("n2_score done")
X["n3_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,3]]]), axis = 1)
print("n3_score done")
X["n4_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,4]]]), axis = 1)
print("n4_score done")
X["n5_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,5]]]), axis = 1)
print("n5_score done")

X

(18958, 12)
n1_cd done
n2_cd done
n3_cd done
n1_ed done
n2_ed done
n3_ed done
n1_score done
n2_score done
n3_score done


,x1,x2,x3,x4,y1,y2,y3,y4,is_key,text_score,...,index,n1_cd,n2_cd,n3_cd,n1_ed,n2_ed,n3_ed,n1_score,n2_score,n3_score
0,-0.437662,-0.612695,-0.613019,-0.437559,-0.903574,-0.903255,-0.915244,-0.915566,-0.571625,-0.602252,...,-1.737075,0.999662,0.998778,0.997921,0.097824,0.188394,0.265695,-0.602252,-0.602252,-0.566790
1,-0.431512,-0.612695,-0.613019,-0.431408,-0.854215,-0.853892,-0.867180,-0.867512,-0.571625,-0.602252,...,-1.737075,0.999690,0.999678,0.998820,0.090802,0.098076,0.185460,-0.602252,-0.602252,-0.602252
2,-0.431512,-0.607202,-0.607528,-0.431408,-0.808381,-0.808054,-0.822550,-0.822891,-0.571625,-0.602252,...,-1.737075,0.999690,0.999666,0.998780,0.090802,0.094887,0.181173,-0.602252,-0.602252,-0.602252
3,-0.425361,-0.601709,-0.602036,-0.425258,-0.759022,-0.758691,-0.777920,-0.778270,-0.571625,-0.602252,...,-1.737075,0.999644,0.999666,0.998758,0.087400,0.094887,0.177649,-0.602252,-0.602252,-0.602252
4,-0.431512,-0.601709,-0.602036,-0.431408,-0.716714,-0.716379,-0.733290,-0.733648,-0.571625,-0.602252,...,-1.737075,0.999644,0.999650,0.998787,0.087400,0.091298,0.179443,-0.602252,-0.602252,-0.658786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18953,1.856431,1.883818,1.885728,1.856488,1.019670,1.020160,1.043331,1.037475,-0.571625,1.596951,...,1.706051,0.991131,0.985762,0.986925,0.662193,0.868813,1.011973,1.810940,1.038765,1.628571
18954,-0.382308,-0.214461,-0.217613,-0.382206,1.285857,1.289896,1.307678,1.303487,1.749399,-0.806596,...,1.706051,0.997310,0.974821,0.975201,0.358333,0.967054,0.923915,-0.867383,-0.534532,-0.533188
18955,1.856431,1.867339,1.874744,1.856488,1.294671,1.295185,1.307678,1.308635,-0.571625,1.038765,...,1.706051,0.994163,0.985762,0.986341,0.689800,0.868813,0.879029,1.628571,1.596951,1.021777
18956,-0.400760,-0.052421,-0.050115,-0.400657,1.421594,1.425646,1.439852,1.437351,1.749399,0.062839,...,1.706051,0.993435,0.979740,0.973088,0.514323,0.870815,0.976549,0.379525,0.131017,-0.806596


In [19]:
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras import layers

In [21]:
#data = receipts_df[['x1','x2','x3','x4','y1','y2','y3','y4','is_key','text_score','category_idx']]
data = X

data['category_idx']=receipts_df['category_idx'];

dnnModel = models.Sequential()
# Layer 1 = input layer
# specify the input size in the first layer.

dnnModel.add(layers.Dense(60, activation='elu', input_shape= (20,)))

# Layer 2 = hidden layer 
#dnnModel.add(layers.Dense(60, activation='relu'))

# Layer 3 = hidden layer 
#dnnModel.add(layers.Dense(30, activation='linear'))

# Layer 4 = output layer
dnnModel.add(layers.Dense(len(category2idx), activation='sigmoid'))

dnnModel.summary()
dnnModel.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'] )
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
y = data.pop('category_idx')
X = data

import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
x = X.values #returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = preprocessing.StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
X = pd.DataFrame(x_scaled)

X_train,X_test,y_train,y_test = train_test_split(X.index,y.index,test_size=0.2)
xtrain=X.iloc[X_train]
xtest=X.iloc[X_test]
y_train=y.iloc[y_train]
y_test=y.iloc[y_test]

from imblearn.over_sampling import RandomOverSampler
from collections import Counter
print('Before Resampled dataset shape' )
print( y_train.shape)
sm = RandomOverSampler('all',random_state=420)
xtrain, y_train = sm.fit_resample(xtrain, y_train)
print(' Resampled dataset shape' )
print( y_train.shape)

ytrain = np.zeros((len(y_train), len(idx2category.values())))
ytest = np.zeros((len(y_test), len(idx2category.values())))
for i in range(len(y_train)):
    #print(y_train.values[i])
    ytrain[i][y_train.values[i]]=1
    
for i in range(len(y_test)):
    #print(y_train.values[i])
    ytrain[i][y_test.values[i]]=1

xtrain=tf.convert_to_tensor(xtrain)

ytraintens=tf.convert_to_tensor(ytrain)

h  = dnnModel.fit( xtrain, ytraintens, epochs=25, batch_size=10)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 60)                1260      
_________________________________________________________________
dense_3 (Dense)              (None, 30)                1830      
Total params: 3,090
Trainable params: 3,090
Non-trainable params: 0
_________________________________________________________________
Before Resampled dataset shape
(15166,)
 Resampled dataset shape
(126390,)


C:\Users\kalus\anaconda3\lib\site-packages\imblearn\utils\_validation.py:638: FutureWarning: Pass sampling_strategy=all as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


Epoch 1/25
12639/12639 [==============================] - 10s 802us/step - loss: 1.6145 - accuracy: 0.5258
Epoch 2/25
12639/12639 [==============================] - 10s 814us/step - loss: 1.2726 - accuracy: 0.6213
Epoch 3/25
12639/12639 [==============================] - 10s 794us/step - loss: 1.1870 - accuracy: 0.6489
Epoch 4/25
12639/12639 [==============================] - 10s 800us/step - loss: 1.1334 - accuracy: 0.6667
Epoch 5/25
12639/12639 [==============================] - 10s 806us/step - loss: 1.0951 - accuracy: 0.6800
Epoch 6/25
12639/12639 [==============================] - 10s 809us/step - loss: 1.0652 - accuracy: 0.6889
Epoch 7/25
12639/12639 [==============================] - 10s 813us/step - loss: 1.0419 - accuracy: 0.6974
Epoch 8/25
12639/12639 [==============================] - 10s 813us/step - loss: 1.0233 - accuracy: 0.7039
Epoch 9/25
12639/12639 [==============================] - 10s 811us/step - loss: 1.0078 - accuracy: 0.7080
Epoch 10/25
12639/12639 [============